In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from neural_network import NeuralNetworkModel, ChessDataset, ChessEvaluationNeuralNetwork

In [20]:
def eval_to_output(eval: str):
    if eval.startswith('#+'):
        return 10000.0
    elif eval.startswith('#-'):
        return -10000.0
    else:
        return float(eval)

In [21]:
df = pd.read_csv("../data/chess_data.csv")

X = df['FEN'].apply(NeuralNetworkModel.fen_to_feature_array).tolist()
X = np.array(X, dtype=np.float32)
y = df['Evaluation'].apply(eval_to_output).astype(np.float32).values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_dataset = ChessDataset(X_train, y_train)
test_dataset = ChessDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [22]:
model = ChessEvaluationNeuralNetwork(input_size=X.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1, Loss: 927398.2552
Epoch 2, Loss: 846123.5489
Epoch 3, Loss: 732739.8290
Epoch 4, Loss: 616434.1369
Epoch 5, Loss: 501248.8284
Epoch 6, Loss: 418301.9346
Epoch 7, Loss: 368073.9103
Epoch 8, Loss: 328062.8971
Epoch 9, Loss: 306814.2616
Epoch 10, Loss: 280485.7327
Epoch 11, Loss: 270302.3144
Epoch 12, Loss: 250923.2870
Epoch 13, Loss: 241545.1659
Epoch 14, Loss: 219909.5801
Epoch 15, Loss: 212324.8529
Epoch 16, Loss: 205371.1747
Epoch 17, Loss: 188099.4131
Epoch 18, Loss: 181661.3117
Epoch 19, Loss: 166485.1304
Epoch 20, Loss: 162593.5315


In [23]:
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
rmse_error = np.sqrt(avg_test_loss)
print(f"\nTest RMSE: {rmse_error:.4f}")


Test RMSE: 611.4586


In [24]:
torch.save(model.state_dict(), 'neural_network.pth')